# Vaccination center scraper

We want to scrap information about vaccination centers from https://ockoreport.uzis.cz/

Currently we present only example for 'Fakultní Nemocnice Bulovka'

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# import re
import numpy as np

#### List of regions

In [89]:
regions = ["Hlavní+město+Praha",
        "Jihočeský",
        "Jihomoravský",
        "Karlovarský",
        "Liberecký",
        "Moravskoslezský",
        "Olomoucký",
        "Pardubický",
        "Plzeňský",
        "Středočeský",
        "Ústecký", 
        "Vysočina", 
        "Zlínský",
        "Královehradecký"]

#### Getting data from https://ockoreport.uzis.cz/ for all the regions

In [94]:
def get_regions_om(regs) -> list:
    """
    Scrapes a link for every vaccination point in CR.
    Input : A list of regions' names
    Output : A list of links
    """
    links = []
    for kraj in regs:
        centers = BeautifulSoup(requests.get(f"https://ockoreport.uzis.cz/Report/FilterRegion?filterRegion={kraj}").text.strip()).select('div[class=list] >a')
        links += ["https://ockoreport.uzis.cz" + link['href'] for link in centers]
    return links

#### requesting data from page for 'Fakultní Nemocnice Bulovka'

In [95]:
links = get_regions_om(regions)
bulovka_html = requests.get(links[9]).text.strip()

In [96]:
bulovka_soup = BeautifulSoup(bulovka_html)
# bulovka_soup

#### obtaining name of vaccination center

In [97]:
vacc_center_name = bulovka_soup.select_one('div[class=detail] > div[class=detail__main] > h1').text
vacc_center_name

'Fakultní Nemocnice Bulovka, Budínova 67/2, Praha 8 - Libeň'

#### Obtaining basic info about the center

In [98]:
bulovka_table_info = bulovka_soup.select('div[class=info] > table > tbody > tr')
bulovka_table_info

[<tr>
 <td>Adresa</td>
 <td>Budínova 67/2, 18000 Praha 8 - Libeň</td>
 </tr>,
 <tr>
 <td>Upřesnění polohy</td>
 <td>Budova č. 7, 2. patro</td>
 </tr>,
 <tr>
 <td>Telefon</td>
 <td><a href="tel:734397747">734397747</a></td>
 </tr>,
 <tr>
 <td>Email</td>
 <td><a href="mailto:ockovanicovid@bulovka.cz">ockovanicovid@bulovka.cz</a></td>
 </tr>,
 <tr>
 <td>Poznámka</td>
 <td>Očkovací centrum Fakultní nemocnice Bulovka se nachází v budově č. 7 pavilon infekce. 
 Objednávání probíhá přes stránky registrace.mzcr.cz</td>
 </tr>,
 <tr>
 <td>Denní kapacita očkování</td>
 <td>510</td>
 </tr>]

In [99]:
bulovka_info = {i.select_one('td:nth-child(1)').text : i.select_one('td:nth-child(2)').text.replace("\r\n", "")
                for i in bulovka_table_info}
bulovka_info

{'Adresa': 'Budínova 67/2, 18000 Praha 8 - Libeň',
 'Upřesnění polohy': 'Budova č. 7, 2. patro',
 'Telefon': '734397747',
 'Email': 'ockovanicovid@bulovka.cz',
 'Poznámka': 'Očkovací centrum Fakultní nemocnice Bulovka se nachází v budově č. 7 pavilon infekce. Objednávání probíhá přes stránky registrace.mzcr.cz',
 'Denní kapacita očkování': '510'}

#### Obtaining opening hours for the given center

In [100]:
bulovka_open_table = bulovka_soup.select('div[class=detail__aside] > div[class=opening] > table > tbody > tr')
bulovka_open_table

[<tr>
 <td>Pondělí</td>
 <td>
 <p>07:00    - 15:00   </p>
 </td>
 </tr>,
 <tr>
 <td>Úterý</td>
 <td>
 <p>07:00    - 15:00   </p>
 </td>
 </tr>,
 <tr>
 <td>Středa</td>
 <td>
 <p>07:00    - 15:00   </p>
 </td>
 </tr>,
 <tr>
 <td>Čtvrtek</td>
 <td>
 <p>07:00    - 15:00   </p>
 </td>
 </tr>,
 <tr>
 <td>Pátek</td>
 <td>
 <p>07:00    - 15:00   </p>
 </td>
 </tr>,
 <tr>
 <td>Sobota</td>
 <td>
 <p>07:00    - 19:00   </p>
 </td>
 </tr>,
 <tr>
 <td>Neděle</td>
 <td>
 <p>07:00    - 19:00   </p>
 </td>
 </tr>]

In [101]:
bulovka_open_hours = {i.select_one('td:nth-child(1)').text : i.select_one('td:nth-child(2)').text.strip().replace(" ", "")
                     for i in bulovka_open_table}
bulovka_open_hours

{'Pondělí': '07:00-15:00',
 'Úterý': '07:00-15:00',
 'Středa': '07:00-15:00',
 'Čtvrtek': '07:00-15:00',
 'Pátek': '07:00-15:00',
 'Sobota': '07:00-19:00',
 'Neděle': '07:00-19:00'}

#### adding everything into the DataFrame

In [72]:
df_bulovka = pd.DataFrame(data=[vacc_center_name], index=["Očkovací místo"], columns=["info"])

df_info = pd.DataFrame.from_dict(bulovka_info, orient='index', columns=['info'])
df_open_hours = pd.DataFrame.from_dict(bulovka_open_hours, orient='index', columns=['info'])

df_bulovka = df_bulovka.append(df_info).append(df_open_hours)
df_bulovka

,info
Očkovací místo,Sportovní hala Gymnázia Chotěboř
Adresa,"Jiráskova 637, Chotěboř"
Upřesnění polohy,Sportovní hala Gymnázia Chotěboř
Telefon,
Email,
Poznámka,"Očkovací centrum Chotěboř, se nachází v budově..."
Denní kapacita očkování,120
Pondělí,Zavřeno
Úterý,12:00-20:00
Středa,12:00-20:00
